In [2]:
import os
import json
import pandas as pd
import traceback
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_community.callbacks.manager import get_openai_callback
import PyPDF2

c:\Users\nadir\Desktop\MCQ\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_base="https://openrouter.ai/api/v1",
    model="openrouter/aurora-alpha",
)


In [4]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [5]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for students in {level} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
### RESPONSE_JSON
{response_json}

"""

In [6]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "level", "response_json"],
    template=TEMPLATE
)

In [7]:
quiz_chain = quiz_generation_prompt | model

In [8]:
dat_path = '../data.txt'

with open(dat_path, 'r') as f:
    text = f.read()
print(text)

Association football, more commonly known as football or in some countries as soccer,[a] is a team sport played between two teams of 11 players who almost exclusively use their feet to propel a ball around a pitch.

The objective of the game is to score more goals than the opposing team by moving the ball beyond the goal line into a rectangular-framed goal defended by the opponent. Traditionally, the game has been played over two 45-minute halves, for a total match time of 90 minutes. With an estimated 250 million players active in over 200 countries and territories, it is the world's most popular sport.

Association football is played in accordance with the Laws of the Game, a set of rules that has been in effect since 1863 and maintained by the IFAB since 1886. The game is played with a football that is 68â€“70 cm (27â€“28 in) in circumference. When the ball is in play, the players mainly use their feet, but may also use any other part of their body, except for their hands or arms, t

In [9]:
result = quiz_chain.invoke({
    "text": text,
    "number": 3,
    "level": "medium",
    "response_json": json.dumps(RESPONSE_JSON)
})

print(result)

content='{"1": {"mcq": "What is the standard duration of an association football match?", "options": {"a": "60 minutes", "b": "90 minutes", "c": "120 minutes", "d": "80 minutes"}, "correct": "b"}, "2": {"mcq": "Which organization governs association football internationally?", "options": {"a": "UEFA", "b": "IFAB", "c": "FIFA", "d": "CONCACAF"}, "correct": "c"}, "3": {"mcq": "Which competition is considered the most prestigious senior international tournament in association football?", "options": {"a": "UEFA Champions League", "b": "FIFA World Cup", "c": "Olympic Games", "d": "Copa America"}, "correct": "b"}}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 174, 'prompt_tokens': 707, 'total_tokens': 881, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None, 'image_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'video_toke

In [20]:
result_dict = json.loads(result.content)
result_dict

{'1': {'mcq': 'What is the standard duration of an association football match?',
  'options': {'a': '60 minutes',
   'b': '90 minutes',
   'c': '120 minutes',
   'd': '80 minutes'},
  'correct': 'b'},
 '2': {'mcq': 'Which organization governs association football internationally?',
  'options': {'a': 'UEFA', 'b': 'IFAB', 'c': 'FIFA', 'd': 'CONCACAF'},
  'correct': 'c'},
 '3': {'mcq': 'Which competition is considered the most prestigious senior international tournament in association football?',
  'options': {'a': 'UEFA Champions League',
   'b': 'FIFA World Cup',
   'c': 'Olympic Games',
   'd': 'Copa America'},
  'correct': 'b'}}

In [21]:
quiz_df = pd.DataFrame([
    {
        "MCQ": v["mcq"],
        "Option A": v["options"]["a"],
        "Option B": v["options"]["b"],
        "Option C": v["options"]["c"],
        "Option D": v["options"]["d"],
        "Correct": v["correct"]
    }
    for v in result_dict.values()
])

quiz_df.to_csv("quiz.csv", index=False)
print(quiz_df)

                                                 MCQ               Option A  \
0  What is the standard duration of an associatio...             60 minutes   
1  Which organization governs association footbal...                   UEFA   
2  Which competition is considered the most prest...  UEFA Champions League   

         Option B       Option C      Option D Correct  
0      90 minutes    120 minutes    80 minutes       b  
1            IFAB           FIFA      CONCACAF       c  
2  FIFA World Cup  Olympic Games  Copa America       b  
